https://devblogs.nvidia.com/deep-learning-self-driving-cars/

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
import sklearn
import cv2

tf.python.control_flow_ops = tf

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

Using TensorFlow backend.


In [3]:
DATA_DIR = '../bclone-data-standard'

In [4]:
import bclone

In [5]:
log_df = bclone.load_log(DATA_DIR)
n_samples = len(log_df) * 3

print('Number of samples (including all three cameras): ', n_samples)

Number of samples (including all three cameras):  24108


In [6]:
gen = bclone.data_generator(DATA_DIR, batch_size=4, controls=['steering'])
a, b = next(gen)
a.shape, b.shape

((12, 160, 320, 3), (12,))

In [7]:
model = Sequential()
model.add(Lambda(lambda x: x / 255. - 0.5, input_shape=(160, 320, 3)))
model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

[How to Use Generators](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/46a70500-493e-4057-a78e-b3075933709d/concepts/b602658e-8a68-44e5-9f0b-dfa746a0cc1a)

In [8]:
#train_gen = bclone.data_generator()

In [9]:
#history = model.fit_generator(train_generator, samples_per_epoch= /
#            len(train_samples), validation_data=validation_generator, /
#            nb_val_samples=len(validation_samples), nb_epoch=3)